# SVI Part III: ELBO 梯度估计

本文讲解 SVI 的 ELBO 优化理论，如何得到 ELBO Gradient Estimators。

回顾：

- 因为 `model = primitive r.v. + deterministic function`, 那么 `guide` 就应该对应的是此公式中 `latent r.v.` 给定部分 `observed r.v.` 的后验分布。
- 在变分自编码器中，用来近似后验分布的 `guide` $q_\phi(z)$ 是局部的，意味着对一个每个样本 $x_i$, `latent r.v.` 的后验分布 $q_\phi(z|x_i)$ 都不相同，我们需要学习与样本个数相同数量的后验分布。

## Setup

我们定义一个 Pyro model $p_{\theta}({\bf x}, {\bf z}) = p_{\theta}({\bf x}|{\bf z}) p_{\theta}({\bf z})$, where ${\bf x}$ 为可观测变量 and ${\bf z}$ 为潜变量。我们还定义一个 Pyro guide $q_{\phi}({\bf z})$（也就是变分分布）。 这里 ${\theta}$ and $\phi$ 分别被成为 model and guide 变分参数. (In particular these are _not_ random variables that call for a Bayesian treatment).

(通过优化 ELBO 来最大化$\log p_{\theta}({\bf x})$) We'd like to maximize the log evidence $\log p_{\theta}({\bf x})$ by maximizing the ELBO (the evidence lower bound) given by 

$${\rm ELBO} \equiv \mathbb{E}_{q_{\phi}({\bf z})} \left [ 
\log p_{\theta}({\bf x}, {\bf z}) - \log q_{\phi}({\bf z})
\right]$$

(使用梯度下降法优化 ELBO) To do this we're going to take (stochastic) gradient steps on the ELBO in the parameter space $\{ \theta, \phi \}$ (有关此方法的早期工作，请参见参考文献[1,2]). So we need to be able to compute unbiased estimates of 

$$\nabla_{\theta,\phi} {\rm ELBO} = \nabla_{\theta,\phi}\mathbb{E}_{q_{\phi}({\bf z})} \left [ 
\log p_{\theta}({\bf x}, {\bf z}) - \log q_{\phi}({\bf z})
\right]$$

对于一般的随机函数 `model()` and `guide()` 如何计算估计上面梯度呢? To simplify notation let's generalize our discussion a bit and ask how we can compute gradients of expectations of an arbitrary cost function $f({\bf z})$. Let's also drop any distinction between $\theta$ and $\phi$. 问题转化成为了计算：

$$\nabla_{\phi}\mathbb{E}_{q_{\phi}({\bf z})} \left [
f_{\phi}({\bf z}) \right]$$

让我们从最简单的情况开始。

## 简单情况: 可重参数化

Suppose that we can reparameterize things such that 

$$\mathbb{E}_{q_{\phi}({\bf z})} \left [f_{\phi}({\bf z}) \right]
=\mathbb{E}_{q({\bf \epsilon})} \left [f_{\phi}(g_{\phi}({\bf \epsilon})) \right]$$

Crucially we've moved all the $\phi$ dependence inside of the expectation; $q({\bf \epsilon})$ is a fixed distribution with no dependence on $\phi$. 这种重新参数化可以适用许多分布（例如正态分布）；更多详情参见参考文献[3]. This kind of reparameterization can be done for many distributions (e.g. the normal distribution); see reference [3] for a discussion. In this case we can pass the gradient straight through the expectation to get

$$\nabla_{\phi}\mathbb{E}_{q({\bf \epsilon})} \left [f_{\phi}(g_{\phi}({\bf \epsilon})) \right]=
\mathbb{E}_{q({\bf \epsilon})} \left [\nabla_{\phi}f_{\phi}(g_{\phi}({\bf \epsilon})) \right]$$

Assuming $f(\cdot)$ and $g(\cdot)$ are sufficiently smooth, we can now get unbiased estimates of the gradient of interest by taking a Monte Carlo estimate of this expectation.

## Tricky case: 不可重参数化


如果我们不能进行上述重新参数化怎么办？不幸的是，许多感兴趣的分布（例如所有离散分布）都是这种情况。在这种情况下，我们的估算器 takes a bit more complicated form.

We begin by expanding the gradient of interest as

$$\nabla_{\phi}\mathbb{E}_{q_{\phi}({\bf z})} \left [
f_{\phi}({\bf z}) \right]= 
\nabla_{\phi} \int d{\bf z} \; q_{\phi}({\bf z}) f_{\phi}({\bf z})$$

使用链式法则得到：

$$ \int d{\bf z} \; \left \{ (\nabla_{\phi}  q_{\phi}({\bf z})) f_{\phi}({\bf z}) + q_{\phi}({\bf z})(\nabla_{\phi} f_{\phi}({\bf z}))\right \} $$

At this point we run into a problem. We know how to generate samples from $q(\cdot)$&mdash;we just run the guide forward&mdash;but $\nabla_{\phi}  q_{\phi}({\bf z})$ isn't even a valid probability density. 因此，我们需要对这个公式进行 massage (也就是简单变形)，使其以期望 w.r.t. $q(\cdot)$ 的形式出现. This is easily done using the identity

$$ \nabla_{\phi}  q_{\phi}({\bf z}) = 
q_{\phi}({\bf z})\nabla_{\phi} \log q_{\phi}({\bf z})$$

which allows us to rewrite the gradient of interest as 

$$\mathbb{E}_{q_{\phi}({\bf z})} \left [
(\nabla_{\phi} \log q_{\phi}({\bf z})) f_{\phi}({\bf z}) + \nabla_{\phi} f_{\phi}({\bf z})\right]$$

This form of the gradient&mdash;variously known as the REINFORCE estimator or the score function estimator or the likelihood ratio estimator&mdash;适用于简单的蒙特卡洛估计。

Note that one way to package this result (which is convenient for implementation) is to introduce a surrogate objective function

$${\rm surrogate \;objective} \equiv
\log q_{\phi}({\bf z}) \overline{f_{\phi}({\bf z})} + f_{\phi}({\bf z})$$  

Here the bar indicates that the term is held constant (i.e. it is not to be differentiated w.r.t. $\phi$). 为了获得（单样本）蒙特卡洛梯度估计，我们对潜变量进行了采样 w.r.t. guide，计算替代目标函数，并进行微分。The result is an unbiased estimate of $\nabla_{\phi}\mathbb{E}_{q_{\phi}({\bf z})} \left [
f_{\phi}({\bf z}) \right]$. In equations:

$$\nabla_{\phi} {\rm ELBO} = \mathbb{E}_{q_{\phi}({\bf z})} \left [ 
\nabla_{\phi} ({\rm surrogate \; objective}) \right]$$


有两个策略可以继续改进, 一个策略是考虑使用特殊结构的损失函数, 另外一个策略是改进梯度方向(像动量法一样).

## 缓解方差问题策略

<small> *Variance 问题：为什么希望做 MLE Deep Learning* </small> 

我们现在有了一个通用方法 for an unbiased gradient estimator of expectations of cost functions. 不幸的是, in the more general case where our $q(\cdot)$ includes non-reparameterizable random variables, 该估计量倾向于具有高方差. Indeed in many cases of interest the variance is so high that the estimator is effectively unusable. 所以我们需要减少方差的策略 (for a discussion see reference [4]). We're going to pursue two strategies.

- The first strategy takes advantage of the particular structure of the cost function $f(\cdot)$. 
- The second strategy effectively introduces a way to reduce variance by using information from previous estimates of 
$\mathbb{E}_{q_{\phi}({\bf z})} [ f_{\phi}({\bf z})]$. As such it is somewhat analogous to using momentum in stochastic gradient descent. 

### Reducing Variance via Dependency Structure

利用概率图结构，也就是概率分布结构减少方差，本部分内容首先介绍其相关理论，然后给出一个 Rao-Blackwellization 例子。

在上面的讨论中 we stuck to a general cost function $f_{\phi}({\bf z})$. We could continue in this vein (the approach we're about to discuss is applicable in the general case) but for concreteness let's zoom back in. 在随机变分推断的过程中，我们感兴趣的 cost function 有如下形式：

$$\log p_{\theta}({\bf x} | {\rm Pa}_p ({\bf x})) +
\sum_i \log p_{\theta}({\bf z}_i | {\rm Pa}_p ({\bf z}_i)) 
- \sum_i \log q_{\phi}({\bf z}_i | {\rm Pa}_q ({\bf z}_i))$$

where we've broken the log ratio $\log p_{\theta}({\bf x}, {\bf z})/q_{\phi}({\bf z})$ into an observation log likelihood piece and a sum over the different latent random variables $\{{\bf z}_i \}$. We've also introduced the notation ${\rm Pa}_p (\cdot)$ and ${\rm Pa}_q (\cdot)$ to denote the parents of a given random variable in the model and in the guide, respectively. 

(读者可能会担心 what the appropriate notion of dependency would be in the case of general stochastic functions; 这里我们仅仅指 regular ol' dependency within a single execution trace). The point is that different terms in the cost function have different dependencies on the random variables $\{ {\bf z}_i \}$ and this is something we can leverage.

简而言之, 对于任何不可重参数化的潜变量 ${\bf z}_i$ the surrogate objective is going to have a term 

$$\log q_{\phi}({\bf z}_i) \overline{f_{\phi}({\bf z})} $$

It turns out that we can remove some of the terms in $\overline{f_{\phi}({\bf z})}$ and still get an unbiased gradient estimator; furthermore, doing so will generally decrease the variance. 具体来说 (see reference [4] for details) we can remove any terms in $\overline{f_{\phi}({\bf z})}$ that are not downstream of the latent variable ${\bf z}_i$ (downstream w.r.t. to the dependency structure of the guide). 请注意，这种一般性的技巧（通常通过分析处理某些随机变量以减少差异）通常被称为 Rao-Blackwellization。

在Pyro中，所有这些逻辑都由`SVI`类自动处理。In particular as long as we use a `TraceGraph_ELBO` loss, Pyro will **keep track of the dependency structure** within the execution traces of the model and guide and **construct a surrogate objective that has all the unnecessary terms removed**:

```python
svi = SVI(model, guide, optimizer, TraceGraph_ELBO())
```

请注意，利用此随机变量依赖关系信息需要额外的计算，所以 `TraceGraph_ELBO` 仅应在模型具有不可重参数化的随机变量的情况下使用； in most applications `Trace_ELBO` suffices.

#### Rao-Blackwellization 例子

Suppose we have a gaussian mixture model with $K$ components. For each data point we: (i) first sample the component distribution $k \in [1,...,K]$; and (ii) observe the data point using the $k^{\rm th}$ component distribution. The simplest way to write down a model of this sort is as follows:

```python
ks = pyro.sample("k", dist.Categorical(probs)
                          .to_event(1))
pyro.sample("obs", dist.Normal(locs[ks], scale)
                       .to_event(1),
            obs=data)
```

Since the user hasn't taken care to mark any of the conditional independencies in the model, the gradient estimator constructed by Pyro's `SVI` class is unable to take advantage of Rao-Blackwellization, with the result that the gradient estimator will tend to suffer from high variance. To address this problem the user needs to explicitly mark the conditional independence. Happily, this is not much work:


```python
# mark conditional independence 
# (assumed to be along the rightmost tensor dimension)
with pyro.plate("foo", data.size(-1)):
    ks = pyro.sample("k", dist.Categorical(probs))
    pyro.sample("obs", dist.Normal(locs[ks], scale),
                obs=data)
```      

That's all there is to it.

#### Aside: Dependency tracking in Pyro 

最后，谈谈 dependency tracking. 在包含任意Python代码的随机函数中track dependency 有些棘手。Pyro中当前实现的方法类似于 WebPPL 中使用的方法(cf. reference [5]). Briefly, a conservative notion of dependency is used that relies on sequential ordering. If random variable ${\bf z}_2$ follows ${\bf z}_1$ in a given stochastic function then ${\bf z}_2$ _may be_ dependent on ${\bf z}_1$ and therefore _is_ assumed to be dependent. To mitigate the overly coarse conclusions that can be drawn by this kind of dependency tracking, Pyro includes constructs for declaring things as independent, namely `plate` and `markov` ([参见上一教程](svi_part_ii.ipynb)). 

For use cases with non-reparameterizable variables, it is therefore important for the user to make use of these constructs (when applicable) to take full advantage of the variance reduction provided by `SVI`. In some cases it may also pay to consider reordering random variables within a stochastic function (if possible). 还值得注意的是，我们希望在将来的Pyro版本中添加更精细的 dependency tracking notations.

### Reducing Variance with Data-Dependent Baselines

减少方差的第二种策略 in our ELBO gradient estimator goes under the name of baselines (see e.g. reference [6], 类似于动量梯度下降法). It actually makes use of the same bit of math that underlies the variance reduction strategy discussed above, except now instead of removing terms we're going to add terms. 基本上，instead of removing terms with zero expectation that tend to _contribute_ to the variance，我们将增添 specially chosen terms with zero expectation 以减小方差。因此，这是一个控制变量策略。

更详细地说，the idea is to take advantage of the fact that for any constant $b$, the following identity holds

$$\mathbb{E}_{q_{\phi}({\bf z})} \left [\nabla_{\phi}
(\log q_{\phi}({\bf z}) \times b) \right]=0$$

This follows since $q(\cdot)$ is normalized:

$$\mathbb{E}_{q_{\phi}({\bf z})} \left [\nabla_{\phi}
\log q_{\phi}({\bf z}) \right]=
 \int \!d{\bf z} \; q_{\phi}({\bf z}) \nabla_{\phi}
\log q_{\phi}({\bf z})=
 \int \! d{\bf z} \; \nabla_{\phi} q_{\phi}({\bf z})=
\nabla_{\phi} \int \! d{\bf z} \;  q_{\phi}({\bf z})=\nabla_{\phi} 1 = 0$$

What this means is that we can replace any term

$$\log q_{\phi}({\bf z}_i) \overline{f_{\phi}({\bf z})} $$

in our surrogate objective with

$$\log q_{\phi}({\bf z}_i) \left(\overline{f_{\phi}({\bf z})}-b\right) $$

Doing so doesn't affect the mean of our gradient estimator but it does affect the variance. If we choose $b$ wisely, we can hope to reduce the variance. 实际上，$b$ 不必是常数：it can depend on any of the random choices upstream (or sidestream) of ${\bf z}_i$.

现在我们已经了解 Reducing Variance with Data-Dependent Baselines 的相关理论，接下来我们看看具体如何构造和使用 Pyro baselines.

### Baselines in Pyro


用户可以通过多种方式 instruct Pyro to use baselines in the context of stochastic variational inference. Since baselines can be attached to any non-reparameterizable random variable, the current baseline interface is at the level of the `pyro.sample` statement. In particular the baseline interface makes use of an argument `baseline`, which is a dictionary that specifies baseline options. 
请注意，仅在 `guide` 中（而不是在 `model` 中）specify baselines for sample statements 是有意义的。

#### Decaying Average Baseline

最简单的 baseline 是根据 $\overline{f_{\phi}({\bf z})}$ 的最近样本的running average 构建的。在 Pyro 中，可以按以下方式调用这种 baseline：

```python
z = pyro.sample("z", dist.Bernoulli(...), 
                infer=dict(baseline={'use_decaying_avg_baseline': True,
                                     'baseline_beta': 0.95}))
```

The optional argument `baseline_beta` specifies the decay rate of the decaying average (default value: `0.90`).

#### Neural Baselines 

在某些情况下，decaying average baseline 效果很好。在其他情况下，using a baseline that depends on upstream randomness is crucial for getting good variance reduction. 构造这样的 baseline 的一种有效的方法是使用神经网络 that can be adapted during the course of learning. Pyro提供了两种specify such a baseline 的方法(for an extended example see the [AIR tutorial](air.ipynb)).

First the user needs to decide what inputs the baseline is going to consume (e.g. the current datapoint under consideration or the previously sampled random variable). Then the user needs to construct a `nn.Module` that encapsulates the baseline computation. This might look something like

```python
class BaselineNN(nn.Module):
    def __init__(self, dim_input, dim_hidden):
        super().__init__()
        self.linear = nn.Linear(dim_input, dim_hidden)
        # ... finish initialization ...

    def forward(self, x):
        hidden = self.linear(x)
        # ... do more computations ...
        return baseline
```

Then, assuming the BaselineNN object `baseline_module` has been initialized somewhere else, in the guide we'll have something like

```python
def guide(x):  # here x is the current mini-batch of data
    pyro.module("my_baseline", baseline_module)
    # ... other computations ...
    z = pyro.sample("z", dist.Bernoulli(...), 
                    infer=dict(baseline={'nn_baseline': baseline_module,
                                         'nn_baseline_input': x}))
```

Here the argument `nn_baseline` tells Pyro which `nn.Module` to use to construct the baseline. On the backend the argument `nn_baseline_input` is fed into the forward method of the module to compute the baseline $b$. Note that the baseline module needs to be registered with Pyro with a `pyro.module` call so that Pyro is aware of the trainable parameters within the module.

Under the hood Pyro constructs a loss of the form 

$${\rm baseline\; loss} \equiv\left(\overline{f_{\phi}({\bf z})} - b  \right)^2$$

which is used to adapt the parameters of the neural network. 没有定理表明这是在这种情况下使用的最佳损失函数(it's not)，但实际上它可以很好地工作。 Just as for the decaying average baseline, the idea is that a baseline that can track the mean $\overline{f_{\phi}({\bf z})}$ will help reduce the variance. Under the hood `SVI` takes one step on the baseline loss in conjunction with a step on the ELBO. 

注意，在实践中，使用不同的学习超参数集可能很重要(e.g. a higher learning rate) for baseline parameters. In Pyro this can be done as follows:

```python
def per_param_args(module_name, param_name):
    if 'baseline' in param_name or 'baseline' in module_name:
        return {"lr": 0.010}
    else:
        return {"lr": 0.001}
    
optimizer = optim.Adam(per_param_args)
```

请注意，为了使整个过程正确，应仅通过 baseline 损失来优化 baseline 参数。同样，`model` 和 `guide` 参数只能通过ELBO进行优化。 To ensure that this is the case under the hood `SVI` detaches the baseline $b$ that enters the ELBO from the autograd graph.  Also, since the inputs to the neural baseline may depend on the parameters of the model and guide, the inputs are also detached from the autograd graph before they are fed into the neural network. 

最后，还有一种供用户 specify a neural baseline 的替代方法。只需使用参数  `baseline_value`:

```python
b = # do baseline computation
z = pyro.sample("z", dist.Bernoulli(...), 
                infer=dict(baseline={'baseline_value': b}))
```

This works as above, except in this case it's the user's responsibility to make sure that any autograd tape connecting $b$ to the parameters of the model and guide has been cut. Or to say the same thing in language more familiar to PyTorch users, any inputs to $b$ that depend on $\theta$ or $\phi$ need to be detached from the autograd graph with `detach()` statements.

### 端对端 Baseline 例子

回想一下，在[第一个SVI教程](svi_part_i.ipynb)中，我们考虑了掷硬币的bernoulli-beta 模型。 因为 beta 随机变量不可重参数化(or rather not easily reparameterizable)，相应的 ELBO 梯度 can be quite noisy. In that context we dealt with this problem by using a Beta distribution that provides (approximate) reparameterized gradients. 
在这里，我们展示了简单的 decaying average baseline 如何减少方差in the case where the Beta distribution is treated as non-reparameterized (so that the ELBO gradient estimator is of the score function type). 在此过程中，我们还使用 `plate` 以完全向量化的方式编写模型。

与直接比较梯度方差不同，我们将了解SVI收敛需要多少步骤。Recall that for this particular model (because of conjugacy) we can compute the exact posterior. So to assess the utility of baselines in this context, 我们设置了以下简单实验. 

We initialize the guide at a specified set of variational parameters. We then do SVI until the variational parameters have gotten to within a fixed tolerance of the parameters of the exact posterior. We do this both with and without the decaying average baseline.然后，我们比较两种情况下所需的梯度步数。下面是完整的代码：

(_Since apart from the use of_ `plate` _and_ `use_decaying_avg_baseline`, _this code is very similar to the code in parts I and II of the SVI tutorial, we're not going to go through the code line by line._ )

In [2]:
import os
import torch
import torch.distributions.constraints as constraints
import pyro
import pyro.distributions as dist
# Pyro also has a reparameterized Beta distribution so we import
# the non-reparameterized version to make our point
from pyro.distributions.testing.fakes import NonreparameterizedBeta
import pyro.optim as optim
from pyro.infer import SVI, TraceGraph_ELBO
import sys

# enable validation (e.g. validate parameters of distributions)
assert pyro.__version__.startswith('1.3.0')
pyro.enable_validation(True)

# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)
max_steps = 2 if smoke_test else 10000


def param_abs_error(name, target):
    return torch.sum(torch.abs(target - pyro.param(name))).item()


class BernoulliBetaExample:
    def __init__(self, max_steps):
        # the maximum number of inference steps we do
        self.max_steps = max_steps
        # the two hyperparameters for the beta prior
        self.alpha0 = 10.0
        self.beta0 = 10.0
        # the dataset consists of six 1s and four 0s
        self.data = torch.zeros(10)
        self.data[0:6] = torch.ones(6)
        self.n_data = self.data.size(0)
        # compute the alpha parameter of the exact beta posterior
        self.alpha_n = self.data.sum() + self.alpha0
        # compute the beta parameter of the exact beta posterior
        self.beta_n = - self.data.sum() + torch.tensor(self.beta0 + self.n_data)
        # initial values of the two variational parameters
        self.alpha_q_0 = 15.0
        self.beta_q_0 = 15.0

    def model(self, use_decaying_avg_baseline):
        # sample `latent_fairness` from the beta prior
        f = pyro.sample("latent_fairness", dist.Beta(self.alpha0, self.beta0))
        # use plate to indicate that the observations are
        # conditionally independent given f and get vectorization
        with pyro.plate("data_plate"):
            # observe all ten datapoints using the bernoulli likelihood
            pyro.sample("obs", dist.Bernoulli(f), obs=self.data)

    def guide(self, use_decaying_avg_baseline):
        # register the two variational parameters with pyro
        alpha_q = pyro.param("alpha_q", torch.tensor(self.alpha_q_0),
                             constraint=constraints.positive)
        beta_q = pyro.param("beta_q", torch.tensor(self.beta_q_0),
                            constraint=constraints.positive)
        # sample f from the beta variational distribution
        baseline_dict = {'use_decaying_avg_baseline': use_decaying_avg_baseline,
                         'baseline_beta': 0.90}
        # note that the baseline_dict specifies whether we're using
        # decaying average baselines or not
        pyro.sample("latent_fairness", NonreparameterizedBeta(alpha_q, beta_q),
                    infer=dict(baseline=baseline_dict))

    def do_inference(self, use_decaying_avg_baseline, tolerance=0.80):
        # clear the param store in case we're in a REPL
        pyro.clear_param_store()
        # setup the optimizer and the inference algorithm
        optimizer = optim.Adam({"lr": .0005, "betas": (0.93, 0.999)})
        svi = SVI(self.model, self.guide, optimizer, loss=TraceGraph_ELBO())
        print("Doing inference with use_decaying_avg_baseline=%s" % use_decaying_avg_baseline)

        # do up to this many steps of inference
        for k in range(self.max_steps):
            svi.step(use_decaying_avg_baseline)
            if k % 100 == 0:
                print('.', end='')
                sys.stdout.flush()

            # compute the distance to the parameters of the true posterior
            alpha_error = param_abs_error("alpha_q", self.alpha_n)
            beta_error = param_abs_error("beta_q", self.beta_n)

            # stop inference early if we're close to the true posterior
            if alpha_error < tolerance and beta_error < tolerance:
                break

        print("\nDid %d steps of inference." % k)
        print(("Final absolute errors for the two variational parameters " +
               "were %.4f & %.4f") % (alpha_error, beta_error))

# do the experiment
bbe = BernoulliBetaExample(max_steps=max_steps)
bbe.do_inference(use_decaying_avg_baseline=True)
bbe.do_inference(use_decaying_avg_baseline=False)

Doing inference with use_decaying_avg_baseline=True
.
Did 84 steps of inference.
Final absolute errors for the two variational parameters were 0.7989 & 0.7734
Doing inference with use_decaying_avg_baseline=False
..
Did 194 steps of inference.
Final absolute errors for the two variational parameters were 0.7989 & 0.7394


**Sample output:**
```
Doing inference with use_decaying_avg_baseline=True
....................
Did 1932 steps of inference.
Final absolute errors for the two variational parameters were 0.7997 & 0.0800
Doing inference with use_decaying_avg_baseline=False
..................................................
Did 4908 steps of inference.
Final absolute errors for the two variational parameters were 0.7991 & 0.2532
```


对于此次运行，我们可以看到 baselines 将我们需要执行的SVI步骤数量大致减少了一半。结果是随机的，并且每次运行都会有所不同，但这是令人鼓舞的结果。这是一个非常人造的示例，但是对于某些 `model` and `guide` 对，baselines 可以带来巨大的成功。

## 参考文献

[1] `Automated Variational Inference in Probabilistic Programming`,
<br/>&nbsp;&nbsp;&nbsp;&nbsp;
David Wingate, Theo Weber

[2] `Black Box Variational Inference`,<br/>&nbsp;&nbsp;&nbsp;&nbsp;
Rajesh Ranganath, Sean Gerrish, David M. Blei

[3] `Auto-Encoding Variational Bayes`,<br/>&nbsp;&nbsp;&nbsp;&nbsp;
Diederik P Kingma, Max Welling

[4] `Gradient Estimation Using Stochastic Computation Graphs`,
<br/>&nbsp;&nbsp;&nbsp;&nbsp;
    John Schulman, Nicolas Heess, Theophane Weber, Pieter Abbeel
    
[5] `Deep Amortized Inference for Probabilistic Programs`
<br/>&nbsp;&nbsp;&nbsp;&nbsp;
Daniel Ritchie, Paul Horsfall, Noah D. Goodman

[6] `Neural Variational Inference and Learning in Belief Networks`
<br/>&nbsp;&nbsp;&nbsp;&nbsp;
Andriy Mnih, Karol Gregor